In [2]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

from bs4 import BeautifulSoup
from lxml import etree
import requests
import FinanceDataReader as fdr
from time import sleep

from tqdm.auto import tqdm

In [3]:
# nasdaq_df = fdr.StockListing('NASDAQ')
nasdaq_df = pd.read_csv('./data/nasdaq_list.csv')

In [4]:
def scrape_yahoo_finance(symbol: str):
    for attempt in range(MAX_ATTEMPT):
        try:
            url = f'https://finance.yahoo.com/quote/{symbol}/key-statistics?p={symbol}'
            webpage = requests.get(url=url, headers=HEADERS, timeout=10)
            soup = BeautifulSoup(webpage.content, "html.parser")
            dom = etree.HTML(str(soup))
        except:
            print(symbol, 'soup failed. Retry', attempt)
            continue
        else:
            try:
                market_cap = 0#dom.xpath('//*[@id="Col1-0-KeyStatistics-Proxy"]/section/div[2]/div[1]/div/div/div/div/table/tbody/tr[1]/td[2]')[0].text
                profit_margin = dom.xpath('//*[@id="Col1-0-KeyStatistics-Proxy"]/section/div[2]/div[3]/div/div[2]/div/div/table/tbody/tr[1]/td[2]')[0].text
                operating_margin = dom.xpath('//*[@id="Col1-0-KeyStatistics-Proxy"]/section/div[2]/div[3]/div/div[2]/div/div/table/tbody/tr[2]/td[2]')[0].text
                revenue = dom.xpath('//*[@id="Col1-0-KeyStatistics-Proxy"]/section/div[2]/div[3]/div/div[4]/div/div/table/tbody/tr[1]/td[2]')[0].text
                gross_profit = dom.xpath('//*[@id="Col1-0-KeyStatistics-Proxy"]/section/div[2]/div[3]/div/div[4]/div/div/table/tbody/tr[4]/td[2]')[0].text
            except:
                print(symbol, 'html element failed. Retry', attempt)
                continue
            else:
                # succeed
                return [market_cap, profit_margin, operating_margin, revenue, gross_profit]
    else:
        # totaly failed
        print(symbol, 'Skip')
        return None

In [5]:
HEADERS = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 \
            (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36",
    "Accept-Language": "en-US, en;q=0.5",
}
MAX_ATTEMPT = 10

df = []
# for i in tqdm(range(len(nasdaq_df))):
for i in tqdm(range(5)):
    name = nasdaq_df.loc[i, 'Name']
    symbol = nasdaq_df.loc[i, 'Symbol']
    industry = nasdaq_df.loc[i, 'Industry']

    data = scrape_yahoo_finance(symbol)
    if data is not None:
        df.append([name, symbol, industry].extend(data))

df = pd.DataFrame(df, columns=['Name', 'Symbol', 'Industry', 'MarketCap', 'ProfitMargin', 'OperatingMargin', 'Revenue', 'GrossProfit'])
df.to_csv('./data/nasdaq_full.csv')
df


  0%|          | 0/5 [00:00<?, ?it/s]

AAPL html element failed. Retry 0
AAPL html element failed. Retry 1
AAPL html element failed. Retry 2
AAPL html element failed. Retry 3
AAPL html element failed. Retry 4
AAPL html element failed. Retry 5
AAPL html element failed. Retry 6
AAPL html element failed. Retry 7
AAPL html element failed. Retry 8
AAPL html element failed. Retry 9
AAPL Skip
MSFT html element failed. Retry 0
MSFT html element failed. Retry 1
MSFT html element failed. Retry 2
MSFT html element failed. Retry 3
MSFT html element failed. Retry 4
MSFT html element failed. Retry 5
MSFT html element failed. Retry 6
MSFT html element failed. Retry 7
MSFT html element failed. Retry 8
MSFT html element failed. Retry 9
MSFT Skip
AMZN html element failed. Retry 0
AMZN html element failed. Retry 1
AMZN html element failed. Retry 2
AMZN html element failed. Retry 3
AMZN html element failed. Retry 4
AMZN html element failed. Retry 5
AMZN html element failed. Retry 6
AMZN html element failed. Retry 7
AMZN html element failed. Re

,Name,Symbol,Industry,MarketCap,ProfitMargin,OperatingMargin,Revenue,GrossProfit


In [ ]:
fig = px.bar(
    df[:10],
    x='Industry',
    y='MarketCap',
    # color=,
    # hover_data=,
    barmode='stack'
)